### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f"{url}api-key={nyt_api_key}&fq={filter_query}&begin_date={begin_date}&end-date={end_date}&sort={sort}&fl={field_list}")
print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=RCq87PkrrB3mZPezLXPOJtrOQdCNQ5VV&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&begin_date=20130101&end-date=20230531&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [4]:
# Create an empty list to store the reviews
reviews_list=[]

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    page_url=f'{query_url}&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page {page} has no results.")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/12/21/movies/anyone-but-you-review.html",
        "snippet": "Sydney Sweeney and Glen Powell romp in a rom-com bomb with gratuitous clothes-shedding, played out against beautiful backdrops.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Anyone but You\u2019 Review: Baring Bums in the Land Down Under",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "Adrift on the Tides of Fake Love",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Anyone but You (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
           

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df = pd.json_normalize(reviews_list)

In [7]:
#reviews_list_df.head()
reviews_list_df['headline.main']

0      ‘Anyone but You’ Review: Baring Bums in the La...
1      ‘Fallen Leaves’ Review: Love (and Laughs) Amon...
2                  ‘Radical’ Review: To Sergio With Love
3                  ‘Fingernails’ Review: Love, Factually
4      ‘My Love Affair With Marriage’ Review: A Tale ...
                             ...                        
195    Review: Love in a Cold Climate in ‘The Mountai...
196    Review: In the Documentary ‘Dina,’ a Different...
197    Review: Love of Il Maestro Drives ‘In Search o...
198    Review: ‘Woodpeckers,’ a Tale of Love and Agon...
199    Review: In ‘I Do ... Until I Don’t,’ Love and ...
Name: headline.main, Length: 200, dtype: object

In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_list_df['title']=reviews_list_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_list_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/12/21/movies/anyo...,Sydney Sweeney and Glen Powell romp in a rom-c...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-12-21T14:00:07+0000,542,‘Anyone but You’ Review: Baring Bums in the La...,None,None,Adrift on the Tides of Fake Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Anyone but You
1,https://www.nytimes.com/2023/11/16/movies/fall...,In the latest from the Finnish filmmaker Aki K...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-11-16T18:04:26+0000,873,‘Fallen Leaves’ Review: Love (and Laughs) Amon...,Critic’s pick,None,"Love, and Laughs, Among the Ruins",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Fallen Leaves
2,https://www.nytimes.com/2023/11/02/movies/radi...,A vaguely unconventional teacher galvanizes di...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-11-02T11:00:25+0000,272,‘Radical’ Review: To Sergio With Love,None,None,Radical,None,None,None,By Natalia Winkelman,"[{'firstname': 'Natalia', 'middlename': None, ...",None,Radical
3,https://www.nytimes.com/2023/11/02/movies/fing...,Jessie Buckley and Riz Ahmed play confused lon...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-11-02T11:00:21+0000,525,"‘Fingernails’ Review: Love, Factually",Critic’s Pick,None,If Love Became A Science,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Fingernails
4,https://www.nytimes.com/2023/10/05/movies/my-l...,This animated musical about a young woman’s se...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-10-05T11:00:31+0000,254,‘My Love Affair With Marriage’ Review: A Tale ...,Critic’s Pick,None,My Love Affair With Marriage,None,None,None,By Natalia Winkelman,"[{'firstname': 'Natalia', 'middlename': None, ...",None,My Love Affair With Marriage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/10/05/movies/the-...,Idris Elba and Kate Winslet crash into a mount...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-10-05T11:00:05+0000,272,Review: Love in a Cold Climate in ‘The Mountai...,None,None,The Mountain Between Us,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Mountain Between Us
196,https://www.nytimes.com/2017/10/05/movies/dina...,"An intimate, affecting film about a couple wit...",The New York Times,"[{'name': 'creative_works', 'value': 'Dina (Mo...",2017-10-05T11:00:05+0000,290,"Review: In the Documentary ‘Dina,’ a Different...",None,None,Dina,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Dina,’ a Differently Abled Love Stor"
197,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None,In Search of Fellini
198,https://www.nytimes.com/2017/09/14/movies/wood...,José María Cabral’s drama concerns a male and ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-09-14T10:00:14+0000,242,"Review: ‘Woodpeckers,’ a Tale of Love and Agon...",None,None,Woodpeckers,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,"Woodpeckers,’ a Tale of Love and Agonizing Pen..."


In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df['keywords']=reviews_list_df['keywords'].apply(extract_keywords)


In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles=reviews_list_df['title'].to_list()
titles

['Anyone but You',
 'Fallen Leaves',
 'Radical',
 'Fingernails',
 'My Love Affair With Marriage',
 'Foe',
 'The Creator',
 'My Sailor, My Love',
 'Cassandro',
 'Love at First Sight',
 'We Kill for Love',
 'Choose Love',
 'Before, Now & Then',
 'Love Life',
 'Red, White & Royal Blue',
 'Love in Taipei',
 'The Eternal Memory',
 'Passages',
 'Sublime',
 'Shooting Stars',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Lo

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
details_url='https://api.themoviedb.org/3/movie/'
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list=[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter=0

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print('Sleeping')
        time.sleep(1)
        
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    tmdb_r=requests.get(url+title+tmdb_key_string).json()
        
    try:
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        
        # Get movie id
        movie_id = tmdb_r['results'][0]['id']
        
        # Make a request for a the full movie details
        #detail_query=details_url+str(movie_id)+"?"+tmdb_key_string
        detail_query_f = f"{details_url}{movie_id}?{tmdb_key_string}"
        
        # Execute "GET" request with url
        movie_details=requests.get(detail_query_f).json()
                
        # Extract the genre names into a list
        genre_list=[]
        for index in range(len(movie_details['genres'])):
            for key in movie_details['genres'][index]:
                if key == "name":
                    genre_list.append(movie_details['genres'][index][key])
            

        # Extract the spoken_languages' English name into a list
        spoken_list=[]
        for index in range(len(movie_details['spoken_languages'])):
            for key in movie_details['spoken_languages'][index]:
                if key == 'english_name':
                    spoken_list.append(movie_details['spoken_languages'][index][key])


        # Extract the production_countries' name into a list
        production_list=[]
        for index in range(len(movie_details['production_countries'])):
            for key in movie_details['production_countries'][index]:
                if key == "name":
                    production_list.append(movie_details['production_countries'][index][key])


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({'title': movie_details["title"],
                   'original_title': movie_details["original_title"],
                   'budget': movie_details["budget"],
                   'original_language': movie_details["original_language"],
                   'homepage': movie_details["homepage"],
                   'overview': movie_details["overview"],
                   'popularity': movie_details["popularity"],
                   'runtime': movie_details["runtime"],
                   'revenue': movie_details["revenue"],
                   'release_date': movie_details["release_date"],
                   'vote_average': movie_details["vote_average"],
                   'vote_count': movie_details["vote_count"],
                   'genres': genre_list,
                   'spoken_languages': spoken_list,
                   'production_countries': production_list
                  })
        
        
        # Print out the title that was found
        print(f"Found {title}")
    except Exception as ex:
        print(f"{title} not found")
        


Found Anyone but You
Found Fallen Leaves
Found Radical
Found Fingernails
Found My Love Affair With Marriage
Found Foe
Found The Creator
Found My Sailor, My Love
Found Cassandro
Found Love at First Sight
Found We Kill for Love
Found Choose Love
Found Before, Now & Then
Found Love Life
Found Red, White & Royal Blue
Found Love in Taipei
Found The Eternal Memory
Found Passages
Found Sublime
Found Shooting Stars
Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5],indent=4))

[
    {
        "title": "Anyone But You",
        "original_title": "Anyone But You",
        "budget": 25000000,
        "original_language": "en",
        "homepage": "https://www.anyonebutyou.movie",
        "overview": "After an amazing first date, Bea and Ben\u2019s fiery attraction turns ice cold \u2014 until they find themselves unexpectedly reunited at a destination wedding in Australia. So they do what any two mature adults would do: pretend to be a couple.",
        "popularity": 421.646,
        "runtime": 104,
        "revenue": 100221578,
        "release_date": "2023-12-21",
        "vote_average": 7.179,
        "vote_count": 95,
        "genres": [
            "Comedy",
            "Romance"
        ],
        "spoken_languages": [
            "English"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Fallen Leaves",
        "original_title": "Kuolleet lehdet",
        "budget": 1480000,
      

In [14]:
# Convert the results to a DataFrame
tmdb_df=pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df=pd.merge(reviews_list_df, tmdb_df, on='title', how='inner')

In [17]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix=['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove=["[","]","'",]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column]=merged_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column]= merged_df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
print(merged_df.head())

                                             web_url  \
0  https://www.nytimes.com/2023/11/16/movies/fall...   
1  https://www.nytimes.com/2023/11/02/movies/radi...   
2  https://www.nytimes.com/2023/11/02/movies/fing...   
3  https://www.nytimes.com/2023/10/05/movies/foe-...   
4  https://www.nytimes.com/2023/09/28/movies/the-...   

                                             snippet              source  \
0  In the latest from the Finnish filmmaker Aki K...  The New York Times   
1  A vaguely unconventional teacher galvanizes di...  The New York Times   
2  Jessie Buckley and Riz Ahmed play confused lon...  The New York Times   
3  Saoirse Ronan and Paul Mescal play a farm coup...  The New York Times   
4  In this hectic, futuristic action film, John D...  The New York Times   

                                            keywords  \
0  subject: Movies;persons: Kaurismaki, Aki;creat...   
1  subject: Movies;persons: Derbez, Eugenio;creat...   
2  subject: Movies;persons: Nikou, Chr

In [18]:
# Drop "byline.person" column
del merged_df["byline.person"]

In [19]:
# Delete duplicate rows and reset index
merged_df=merged_df.drop_duplicates().reset_index(drop=True)

In [21]:
# Export data to CSV without the index
merged_df.to_csv('output\collected_data.csv', index=False)